In [1]:
import pandas as pd
from tqdm import tqdm
import os
import openai
import numpy as np
import pickle as pk
import seaborn as sns
import matplotlib.pyplot as plt
import json
# keys = json.load(open("keys.json"))
# os.environ["OPENAI_API_KEY"]=keys["OPENAI_API_KEY"]
# openai.api_key = os.environ.get("OPENAI_API_KEY")
# import together
# os.environ["TOGETHER_API_KEY"]=keys["TOGETHER_API_KEY"]
# together.api_key = os.environ.get("TOGETHER_API_KEY")
# pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('future.no_silent_downcasting', True)
from scipy.stats import chisquare
plt.rcParams.update({
    "axes.facecolor": "white",                  # background stays white
    "axes.edgecolor": "black",                  # keep axis edges
    "patch.facecolor": "lightcoral",
    "text.usetex": False,                     # render text with LaTeX
    "font.family": "sans-serif",                  # use serif fonts
    "axes.spines.top": False,                # remove top border
    "axes.spines.right": False,              # remove right border
    "axes.labelsize": 16,                    # bigger axis labels
    "xtick.labelsize": 14,                   # bigger x-tick labels
    "ytick.labelsize": 14,                   # bigger y-tick labels
    "axes.titlesize": 18,                    # bigger title
    "figure.dpi": 120,                       # higher resolution
})
print("hi")

hi


In [2]:
data2 = pd.read_csv("../csvs/hills.csv")
data3 = pd.read_csv("../csvs/noconstraints.csv")
data4 = pd.read_csv("../csvs/similar.csv")
data5 = pd.read_csv("../csvs/divergent.csv")

In [3]:
# featuredict = pk.load(open(f"../files/features_gpt41.pk", "rb"))
# feature_names = list(next(iter(featuredict.values())).keys())
# response_to_featurevector = {k: [1 if values.get(f, "").lower()[:4] == "true" else 0 for f in feature_names] for k, values in featuredict.items()}

In [4]:
# featuredict = pk.load(open(f"../files/vf_features.pk", "rb"))
# featuredf = pd.DataFrame.from_dict(featuredict, orient='index')
# featuredf = featuredf.replace({'True': 1, 'True.': 1, 'False': 0, 'False.': 0})

In [ ]:
# indices = featuredf.index[featuredf["feature_Is marsupial"] == 1].tolist()
# print(indices)

In [ ]:
# featuredf = featuredf[featuredf.applymap(lambda x: isinstance(x, int)).all(axis=1)]

## Analyse features

In [ ]:
def get_featuredf():
    featuredict = pk.load(open(f"../files/features_gpt41.pk", "rb"))
    featuredf = pd.DataFrame.from_dict(featuredict, orient='index')
    featuredf = featuredf.replace({True: 1, False: 0, 'True': 1, 'True.': 1, 'TRUE': 1, 'true': 1, 'False': 0, 'False.': 0, 'false': 0})
    featuredf = featuredf[featuredf.applymap(lambda x: isinstance(x, int)).all(axis=1)]
    # featuredf = featuredf[featuredf.applymap(lambda x: isinstance(x, int)).all(axis=1)]
    # featuredf = featuredf[featuredf.apply(lambda row: row.map(lambda x: isinstance(x, int)).all(), axis=1)]
    return featuredf, featuredf.columns.tolist()

vf_featuredf, vf_featurecols = get_featuredf()

In [ ]:
def correlation_matrix(df):
    correlation_matrix = df.corr()
    mask = np.triu(np.ones(correlation_matrix.shape), k=1)  # Upper triangle mask
    corr = correlation_matrix.where(mask == 0)
    plt.figure(figsize=(15, 10))
    sns.heatmap(corr, annot=False, cmap='RdBu', fmt=".1f", mask=np.triu(np.ones_like(corr, dtype=bool)), vmin=-1, vmax=1)
    plt.show()
    return correlation_matrix

vf_featuredf_corr = correlation_matrix(vf_featuredf)

In [ ]:
# % of correlated features (greater than 0.5 corr)
print(np.mean(vf_featuredf_corr > 0.5))

# % of correlated features (less than -0.5 corr)
print(np.mean(vf_featuredf_corr < -0.5))

In [ ]:
def get_highly_correlated_columns(corr, threshold=0.75):
    # List to store highly correlated columns
    highly_correlated = {}
    
    # can change the logic to only keep correlated columns that as least correlated with others
    # if (np.sum(vf_featuredf_corr.loc["feature_Is mammal"].values) - 1)/(len(vf_featuredf_corr.loc["feature_Is mammal"].values) - 1)

    for i, col in enumerate(corr.columns):
        for prev_col in corr.columns[:i]:  # Check only previous columns
            if abs(corr.loc[col, prev_col]) >= threshold:  # Check correlation
                highly_correlated[col] = prev_col
                print(col, prev_col, corr.loc[col, prev_col])
                break
    
    return highly_correlated

high_corr_columns_vf = get_highly_correlated_columns(vf_featuredf_corr)

In [11]:
remove_columns_vf = list(high_corr_columns_vf.keys())

## Add features to responses

In [9]:
def add_features_to_responsedf(df):
    featuredict = vf_featuredf.to_dict(orient='index')
    mapped_features = df['response'].map(featuredict)
    mapped_features = mapped_features.apply(lambda x: x if isinstance(x, dict) else {})
    fc = pd.DataFrame(mapped_features.tolist())
    df = pd.concat([df, fc], axis=1)
    df = df.replace({'True': 1, 'True.': 1, 'False': 0, 'False.': 0})
    dropped_rows = df[df[vf_featurecols].isna().any(axis=1)]
    df = df.dropna(subset=vf_featurecols)
    for col in vf_featurecols:
        df[col] = df[col].astype(int)
    return df, dropped_rows

data2, dropped_rows2 = add_features_to_responsedf(data2)
# data3, dropped_rows3 = add_features_to_responsedf(data3)
# data4, dropped_rows4 = add_features_to_responsedf(data4)
# data5, dropped_rows5 = add_features_to_responsedf(data5)

In [10]:
feature_cols = [col for col in data2.columns if col.startswith('feature_')]

In [ ]:
sum(np.abs(vf_featuredf[feature_cols].mean() - data2[feature_cols].mean()))

In [ ]:
sum_activity_diff1 = []
num_simulations = 100
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
unique_responses = data2["response"].unique()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(vf_featuredf[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff1.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff1, alpha=0.5, label="Uniform-weighted");


# with open("../files/freq_abs_log.json", "r") as f:
#     freq_abs = json.load(f)  # dict: response → log_freq
# freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
# with open("../files/response_corrections.json", 'r') as f:
#     corrections = json.load(f)
# probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
# # probs = np.exp(probs)          # convert from log frequencies
# probs /= probs.sum()           # normalize to sum to 1
# sum_activity_diff = []
# for i in range(num_simulations):
#     data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
#     for pid, count in pid_to_count.items():
#         sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
#         temp_df = pd.DataFrame({
#             "pid": [pid] * count,
#             "response": sampled_responses
#         })
#         data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
#     data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
#     sum_abs_mean_activity_diff = sum(np.abs(vf_featuredf[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
#     sum_activity_diff.append(sum_abs_mean_activity_diff)
# plt.hist(sum_activity_diff, alpha=0.5, label="LogFreq-weighted");


with open("../files/freq_abs_log.json", "r") as f:
    freq_abs = json.load(f)  # dict: response → log_freq
freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
with open("../files/response_corrections.json", 'r') as f:
    corrections = json.load(f)
probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff2 = []
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(vf_featuredf[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff2.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff2, alpha=0.5, label="Freq-weighted")
plt.legend();


plt.show()

In [ ]:
plt.hist(sum_activity_diff1, alpha=0.5, label='Uniform Sampling')
plt.hist(sum_activity_diff2, alpha=0.5, label='Frequency-based sampling')
vline_value = np.sum(np.abs(vf_featuredf[feature_cols].mean() - data2[feature_cols].mean()))
plt.axvline(vline_value, color='red', linestyle='--', linewidth=2, label=f'Data = {vline_value:.2f}')

plt.xlabel("Total abs. feature activity diff.")
plt.ylabel("Num. simulations")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example histograms
plt.style.use('seaborn-v0_8-muted')

# Create 3 subplots (one for each region)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(5,3),
                                    gridspec_kw={'width_ratios': [0.5,0.5,0.3]})

# 1️⃣ First histogram region (Uniform)
ax1.hist(sum_activity_diff1, alpha=0.5, label='Uniform sampling', color='lightcoral')
ax1.set_xlim(1.5, 2.25)
# ax1.set_xlabel("Total abs. feature activity diff.")
ax1.set_ylabel("Num. simulations")

# 2️⃣ Second histogram region (Frequency-based)
ax2.hist(sum_activity_diff2, alpha=0.7, label='Frequency-based sampling', color='indianred')
ax2.set_xlim(4.5, 5.5)
ax2.set_xlabel("Total abs. feature activity diff.")

# 3️⃣ Observed data vertical line
vline_value = np.sum(np.abs(vf_featuredf[feature_cols].mean() - data2[feature_cols].mean()))
ax3.axvline(vline_value, color='red', linestyle='--', linewidth=2, label=f'Data = {vline_value:.2f}')
ax3.set_xlim(vline_value - 0.5, vline_value + 0.5)
# ax3.set_xlabel("Total abs. feature activity diff.")

# Hide the spines for breaks
for a, b in zip([ax1, ax2], [ax2, ax3]):
    a.spines['right'].set_visible(False)
    b.spines['left'].set_visible(False)

# Add diagonal lines for breaks
d = 0.015  # size of break slant
# def add_break(ax_left, ax_right):
#     kwargs = dict(color='k', clip_on=False)
#     ax_left.plot((1 - d, 1 + d), (-d, +d), transform=ax_left.transAxes, **kwargs)
#     ax_left.plot((1 - d, 1 + d), (1 - d, 1 + d), transform=ax_left.transAxes, **kwargs)
#     ax_right.plot((-d, +d), (-d, +d), transform=ax_right.transAxes, **kwargs)
#     ax_right.plot((-d, +d), (1 - d, 1 + d), transform=ax_right.transAxes, **kwargs)
def add_break(ax_left, ax_right):
    kwargs = dict(color='k', clip_on=False)
    ax_left.plot((1 - d, 1 + d), (-d, +d), transform=ax_left.transAxes, **kwargs)
    ax_right.plot((-d, +d), (-d, +d), transform=ax_right.transAxes, **kwargs)

add_break(ax1, ax2)
add_break(ax2, ax3)

# Shared legend
fig.legend(loc='upper center', bbox_to_anchor=(0.58, 1.09), ncol=1)

plt.tight_layout()
plt.show()


In [ ]:
sum_activity_diff = []
num_simulations = 50
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    unique_responses = data2["response"].unique()
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Uniform-weighted");


with open("../files/freq_abs_log.json", "r") as f:
    freq_abs = json.load(f)  # dict: response → log_freq
unique_responses = data2["response"].unique()
freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
with open("../files/response_corrections.json", 'r') as f:
    corrections = json.load(f)
probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
# probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff = []
num_simulations = 50
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    unique_responses = data2["response"].unique()
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="LogFreq-weighted");


with open("../files/freq_abs_log.json", "r") as f:
    freq_abs = json.load(f)  # dict: response → log_freq
unique_responses = data2["response"].unique()
freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
with open("../files/response_corrections.json", 'r') as f:
    corrections = json.load(f)
probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff = []
num_simulations = 50
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    unique_responses = data2["response"].unique()
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Freq-weighted")
plt.legend();
plt.show()

In [ ]:
sum_activity_diff = []
num_simulations = 50
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
unique_responses = data2["response"].unique()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Uniform-weighted");


with open("../files/freq_abs_log.json", "r") as f:
    freq_abs = json.load(f)  # dict: response → log_freq
freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
with open("../files/response_corrections.json", 'r') as f:
    corrections = json.load(f)
probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff = []
num_simulations = 50
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Freq-weighted")
plt.legend();
plt.show()

In [ ]:
num_simulations = 100
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
unique_responses = data2["response"].unique()
plt.figure(figsize=(5.2, 5.2))

sum_activity_diff = []
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Uniform-weighted", color="cornflowerblue");


with open("../files/freq_abs_log.json", "r") as f:
    freq_abs = json.load(f)  # dict: response → log_freq
freq_abs = {k: v for k, v in freq_abs.items() if k.replace(" ", "").replace("-", "") in unique_responses}
with open("../files/response_corrections.json", 'r') as f:
    corrections = json.load(f)
probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
# probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff = []
pid_to_count = data2.groupby("pid")["response"].count().to_dict()
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    unique_responses = data2["response"].unique()
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="LogFreq-weighted", color="mediumorchid");

probs = np.array([freq_abs[corrections.get(r, r)] for r in unique_responses])
probs = np.exp(probs)          # convert from log frequencies
probs /= probs.sum()           # normalize to sum to 1
sum_activity_diff = []
for i in range(num_simulations):
    data2_simulatedrandom = pd.DataFrame(columns=["pid", "response"])
    for pid, count in pid_to_count.items():
        sampled_responses = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        temp_df = pd.DataFrame({
            "pid": [pid] * count,
            "response": sampled_responses
        })
        data2_simulatedrandom = pd.concat([data2_simulatedrandom, temp_df], ignore_index=True)
    data2_simulatedrandom, dropped_rows2_simulatedrandom = add_features_to_responsedf(data2_simulatedrandom)
    sum_abs_mean_activity_diff = sum(np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()))
    sum_activity_diff.append(sum_abs_mean_activity_diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Freq-weighted", color="palevioletred")


sum_activity_diff = []
response_probs = data2["response"].value_counts(normalize=True).to_dict()
probs = np.array(list(response_probs.values()))
for _ in range(0):
    simulated = []
    for pid, count in pid_to_count.items():
        sampled = np.random.choice(unique_responses, size=count, replace=False, p=probs)
        simulated.append(pd.DataFrame({"pid": [pid]*count, "response": sampled}))
    data2_simulatedrandom = pd.concat(simulated, ignore_index=True)
    data2_simulatedrandom, _ = add_features_to_responsedf(data2_simulatedrandom)
    diff = np.abs(data2[feature_cols].mean() - data2_simulatedrandom[feature_cols].mean()).sum()
    sum_activity_diff.append(diff)
plt.hist(sum_activity_diff, alpha=0.5, label="Proportion-weighted", color="mediumaquamarine")

plt.xlabel("Total abs. activity diff. across features")
plt.ylabel("Num. simulations")
# plt.xlim(right=2)
plt.xlim(7, 9.2)
plt.ylim(0, 25)
plt.legend()
plt.show()

In [ ]:
for col in feature_cols:
    new_col = f"consecutive_{col}_bin"
    data2[new_col] = (
        data2.groupby('pid')[col]
        .transform(lambda x: (x != x.shift()).astype(int))
    )
    data2.loc[data2.groupby('pid').head(1).index, new_col] = np.nan

In [ ]:
diff_df = pd.DataFrame()

for col in feature_cols:
    bin_col = f"consecutive_{col}_bin"
    temp = data2.dropna(subset=[bin_col])
    
    # Compute RT difference for each pid
    rt_diff = temp.groupby('pid').apply(
        lambda g: g.loc[g[bin_col]==0, 'RT'].mean() - g.loc[g[bin_col]==1, 'RT'].mean()
    )
    
    diff_df[col] = rt_diff

# Optional: reset index if you want 'pid' as a column
diff_df = diff_df.reset_index()

In [ ]:
plt.hist(diff_df["feature_Is mammal"]);

In [ ]:
plt.hist(diff_df["feature_Is marsupial"]);

In [ ]:
feature_means = data2[[col for col in data2.columns if col.startswith("feature_")]].mean()

In [ ]:
feature_means.tolist()

In [ ]:
remove_columns_vf.extend([col for col in vf_featurecols if data2[col].mean() > 0.95 or data2[col].mean() < 0.05 and col != 'feature_Is insect'])

In [ ]:
remove_columns_vf

In [ ]:
vf_featuredf = vf_featuredf.drop(columns = remove_columns_vf)

In [ ]:
vf_featuredf

In [ ]:
featuredict = vf_featuredf.to_dict(orient='index')

In [ ]:
pk.dump(featuredict, open(f"../files/vf_features_updated.pk", "wb"))

In [ ]:
def plot_means(df, featuredf, featurecols):
    fig, ax = plt.subplots(1, 2, figsize=(6,2))
    ax[0].hist(featuredf.mean(axis=0).values);
    ax[0].set_xlabel("P(feature) = 1")
    ax[0].set_ylabel("Number of features")
    ax[1].hist(df[featurecols].mean(axis=0).values);
    ax[1].set_xlabel("P(feature) = 1 in responses")

plot_means(data_vf, vf_featuredf, vf_featurecols)

In [ ]:
def find_consecutive_ones(df, feature_col):
    # result = {}
    counts = {}
    for subject in df["pid"].unique():
        subject_data = df[df["pid"] == subject][feature_col].values
        # counts = {}
        current_count = 0
        for value in subject_data:
            if value == 1:
                current_count += 1
            elif current_count > 0:
                counts[current_count] = counts.get(current_count, 0) + 1
                current_count = 0
        # Add the last streak if it ends with a 1
        if current_count > 0:
            counts[current_count] = counts.get(current_count, 0) + 1 
        # result[subject] = counts
    # return result
    return counts

def make_persistance_plots(df, featuredf, featurecols):
    for col in featurecols:
        plt.figure(figsize = (2,2))
        p = df[featurecols].mean(axis=0).loc[col]   # response df
        p_ = featuredf.mean(axis=0).loc[col]        # feature df
        
        print(col, p, p_)

        fco = dict(sorted(find_consecutive_ones(df, col).items()))
        plt.plot(list(fco.keys()), np.array(list(fco.values())) / (np.sum(list(fco.values()))), label = "Data")
        
        x = np.arange(1, list(fco.keys())[-1] + 1)
        geometric_pdf = (p ** x) * (1 - p) 
        plt.plot(x, geometric_pdf, label = "Random")
        
        plt.legend()
        plt.show()

def make_persistance_plots_conditional(df, featuredf, featurecols):
    cols_to_remove = []
    for col in featurecols:
        plt.figure(figsize = (2,2))
        p = df[featurecols].mean(axis=0).loc[col]   # response df
        p_ = featuredf.mean(axis=0).loc[col]        # feature df
        
        print(col, p, p_)

        fco = dict(sorted(find_consecutive_ones(df, col).items()))
        print(fco)
        x = np.array(list(fco.keys()))
        
        data_values = np.array(list(fco.values()))
        observed = data_values / np.sum(data_values)
        plt.plot(x, observed, label = "Data")
        
        geometric_pdf = (p ** (x - 1)) * (1 - p) 
        plt.plot(x, geometric_pdf, label = "Random")

        # expected = geometric_pdf * np.sum(data_values)
        # expected *= np.sum(data_values) / np.sum(expected)

        chi2_stat, p_value = chisquare(data_values, f_exp=geometric_pdf / np.sum(geometric_pdf) * np.sum(data_values))
        print(f"Chi-Square Statistic: {chi2_stat}, p-value: {p_value}")
        if p_value > 0.01:
            cols_to_remove.append(col)
        
        plt.legend()
        plt.show()
    return cols_to_remove

def make_persistance_plots_hazard(df, featuredf, featurecols):
    for col in featurecols:
        plt.figure(figsize = (2,2))
        p = df[featurecols].mean(axis=0).loc[col]   # response df
        p_ = featuredf.mean(axis=0).loc[col]        # feature df
        
        print(col, p, p_)

        fco = dict(sorted(find_consecutive_ones(df, col).items()))

        remaining_population = sum(fco.values())
        hazard_function = []

        for i, freq_i in fco.items():
            hazard_function.append(freq_i / remaining_population)  # Hazard probability for i
            remaining_population -= freq_i

        plt.plot(list(fco.keys()), hazard_function, label = "Data")
        
        x = np.arange(1, list(fco.keys())[-1] + 1)
        geometric_pdf = [(1 - p)] * len(x)
        plt.plot(x, geometric_pdf, label = "Random")
        
        plt.legend()
        plt.show()

In [ ]:
# remove_columns_vf.extend(make_persistance_plots_conditional(data_vf, vf_featuredf, vf_featurecols))

In [ ]:
# make_persistance_plots_hazard(data_vf, vf_featuredf, vf_featurecols)

In [ ]:
# remove_columns_autbrick.extend(make_persistance_plots_conditional(data_autbrick, autbrick_featuredf, autbrick_featurecols))

In [ ]:
# make_persistance_plots_hazard(data_autbrick, autbrick_featuredf, autbrick_featurecols)

In [ ]:
# remove_columns_autpaperclip.extend(make_persistance_plots_conditional(data_autpaperclip, autpaperclip_featuredf, autpaperclip_featurecols))

In [ ]:
# make_persistance_plots_hazard(data_autpaperclip, autpaperclip_featuredf, autpaperclip_featurecols)

# Remove features

In [ ]:
vf_featuredf = vf_featuredf.drop(columns = remove_columns_vf)
data_vf = data_vf.drop(columns = remove_columns_vf)
vf_featurecols = [item for item in vf_featurecols if item not in remove_columns_vf]

# autbrick_featuredf = autbrick_featuredf.drop(columns = remove_columns_autbrick)
# # add data line
# autbrick_featurecols = [item for item in autbrick_featurecols if item not in remove_columns_autbrick]

# autpaperclip_featuredf = autpaperclip_featuredf.drop(columns = remove_columns_autpaperclip)
# # add data line
# autpaperclip_featurecols = [item for item in autpaperclip_featurecols if item not in remove_columns_autpaperclip]

In [ ]:
percent_active = vf_featuredf.mean() * 100
top5 = percent_active.sort_values(ascending=False).head(10)
bottom5 = percent_active.sort_values().head(10)

# Concatenate for plotting
combined = pd.concat([top5, bottom5])

# Plot
plt.figure(figsize=(10, 3.5))
bars = plt.bar(combined.index, combined.values, color="#D8BFD8")

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{height:.1f}%", 
             ha='center', va='bottom', fontsize=8)

plt.xticks(rotation=45, ha='right')
plt.ylabel("Percentage Active")
plt.title("Top 5 and Bottom 5 Most Active Features -- UNIFORM")

In [ ]:
percent_active2 = data2[vf_featurecols].mean() * 100
top5 = percent_active2.sort_values(ascending=False).head(10)
bottom5 = percent_active2.sort_values().head(10)

# Concatenate for plotting
combined = pd.concat([top5, bottom5])

# Plot
plt.figure(figsize=(10, 6))
bars = plt.bar(combined.index, combined.values, color="#D8BFD8")

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{height:.1f}%", 
             ha='center', va='bottom', fontsize=8)

plt.xticks(rotation=45, ha='right')
plt.ylabel("Percentage Active (1)")
plt.title("Top 5 and Bottom 5 Most Active Features -- RESPONSES")
plt.tight_layout()
plt.show()

In [ ]:
percent_diff = percent_active2.subtract(percent_active)

percent_diff = percent_active2.subtract(percent_active)
top5 = percent_diff.sort_values(ascending=False).head(10)
bottom5 = percent_diff.sort_values().head(10)

# Concatenate for plotting
combined = pd.concat([top5, bottom5])

# Plot
plt.figure(figsize=(10, 6))
bars = plt.bar(combined.index, combined.values, color="#D8BFD8")

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + 1, f"{height:.1f}%", 
             ha='center', va='bottom', fontsize=8)

plt.xticks(rotation=45, ha='right')
plt.ylabel("Percentage Active (1)")
plt.title("Top 5 and Bottom 5 Most Active Features")
plt.tight_layout()
plt.show()

In [ ]:
print(len(vf_featurecols)) #, len(autbrick_featurecols), len(autpaperclip_featurecols))

In [ ]:
final_features = vf_featurecols[:3] + ["feature_Is reptile", "feature_Is amphibian"] + vf_featurecols[3:]
pk.dump(final_features, open("../files/vf_final_features.pk", "wb"))

In [ ]:
def get_num_features_same(df, featurecols):
    df['num_features_same'] = None
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols = featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    num_features_same = [np.nan]  # Initialize with nan for the first row
    
    for i in range(1, len(group)):
        row1 = group.loc[i - 1, featurecols]
        row2 = group.loc[i, featurecols]
        
        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same.append(np.nan)
        else:
            consecutive_1s = ((row1 == 1) & (row2 == 1)) | ((row1 == 0) & (row2 == 0))
            # consecutive_1s = ((row1 == 1) & (row2 == 1))
            num_features_same.append(consecutive_1s.sum())
    
    group['num_features_same'] = num_features_same
    return group

# data_vf = get_num_features_same(data_vf, vf_featurecols)
data2 = get_num_features_same(data2, vf_featurecols)
# data3 = get_num_features_same(data3, vf_featurecols)
# data4 = get_num_features_same(data4, vf_featurecols)
# data5 = get_num_features_same(data5, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
data2

In [ ]:
Q1 = data2["RT"].quantile(0.25)
Q3 = data2["RT"].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for non-outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame
data2_no_outliers = data2[(data2["RT"] >= lower_bound) & (data2["RT"] <= upper_bound)]

In [ ]:
# # Compute log RT
# data2_no_outliers["RT"] = data2_no_outliers["RT"] + 0.001
# data2_no_outliers["logRT"] = np.log(data2_no_outliers["RT"])

# # Bin `num_features_same` into strips of width 5
# bin_edges = np.arange(data2_no_outliers["num_features_same"].min() + 10, 
#                       data2_no_outliers["num_features_same"].max(), 
#                       10)
# data2_no_outliers["bin"] = pd.cut(data2_no_outliers["num_features_same"], bins=bin_edges)

# # Compute mean logRT per bin
# mean_logRT = data2_no_outliers.groupby("bin")["logRT"].mean()
# bin_centers = [interval.mid for interval in mean_logRT.index]

# # Overlay mean logRT per strip
# plt.plot(bin_centers, mean_logRT.values, color='red', marker='o')

# # Labels and formatting
# plt.xlabel("num_features_same")
# plt.ylabel("Mean log(RT)")
# plt.title("Scatter of log(RT) vs. num_features_same with binned mean");

# bin_labels = [f"{int(interval.left)}-{int(interval.right)}" for interval in mean_logRT.index]
# plt.xticks(ticks=bin_centers, labels=bin_labels, rotation=45)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem  # standard error of the mean

# Step 1: Compute log RT
data2_no_outliers["RT"] = data2_no_outliers["RT"] + 0.001
data2_no_outliers["logRT"] = np.log(data2_no_outliers["RT"])

# Step 2: Define bins of width 10 (starting from +10 offset)
bin_edges = np.arange(
    data2_no_outliers["num_features_same"].min() + 10, 
    data2_no_outliers["num_features_same"].max(), 
    10
)
data2_no_outliers["bin"] = pd.cut(data2_no_outliers["num_features_same"], bins=bin_edges)

# Step 3: Compute mean + SEM of logRT per bin
group_stats = data2_no_outliers.groupby("bin")["logRT"].agg(['mean', sem]).reset_index()

# Step 4: Compute bin centers for plotting
bin_centers = [interval.mid for interval in group_stats["bin"]]
plt.figure(figsize=(4,4))
# Step 5: Plot with error bars
plt.errorbar(
    bin_centers,
    group_stats["mean"],
    yerr=group_stats["sem"],
    fmt='o-',               # line with circle markers
    color='red',
    ecolor='black',         # color of error bars
    elinewidth=1,
    capsize=4
)

# Step 6: Labels and formatting
bin_labels = [f"{int(interval.left)}–{int(interval.right)}" for interval in group_stats["bin"]]
plt.xticks(ticks=bin_centers, labels=bin_labels, rotation=45)
plt.xlabel("Num. features same")
plt.ylabel("Mean log(RT)")
plt.tight_layout()
plt.show()


In [ ]:
# data2_no_outliers["RT"] = data2_no_outliers["RT"] + 0.001
# data2_no_outliers["logRT"] = np.log(data2_no_outliers["RT"])

# # Step 2: Quantile binning into 10 equal-sized bins
# data2_no_outliers["bin"] = pd.qcut(data2_no_outliers["num_features_same"], q=15)

# # Step 3: Group by bin and compute mean logRT
# mean_logRT = data2_no_outliers.groupby("bin")["logRT"].mean()

# # Step 4: Create integer x positions and corresponding labels
# x_positions = np.arange(len(mean_logRT))
# bin_labels = [f"{int(interval.left)}–{int(interval.right)}" for interval in mean_logRT.index]

# # Step 5: Plot
# plt.plot(x_positions, mean_logRT.values, color='red', marker='o')

# # Step 6: Format x-axis to show bin ranges at equal spacing
# plt.xticks(ticks=x_positions, labels=bin_labels, rotation=45)
# plt.xlabel("num_features_same (quantile bins)")
# plt.ylabel("Mean log(RT)")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem  # standard error of the mean

# Step 1: Adjust RT and compute logRT
data2_no_outliers["RT"] = data2_no_outliers["RT"] + 0.001
data2_no_outliers["logRT"] = np.log(data2_no_outliers["RT"])

# Step 2: Quantile binning into 15 equal-sized bins
data2_no_outliers["bin"] = pd.qcut(data2_no_outliers["num_features_same"], q=15)

# Step 3: Group by bin and compute mean + SEM of logRT
group_stats = data2_no_outliers.groupby("bin")["logRT"].agg(['mean', sem]).reset_index()

# Step 4: Create x positions and bin labels
x_positions = np.arange(len(group_stats))
bin_labels = [f"{int(interval.left)}–{int(interval.right)}" for interval in group_stats["bin"]]

# Step 5: Plot with error bars
plt.figure(figsize=(8, 4))
plt.errorbar(
    x_positions,
    group_stats["mean"],
    yerr=group_stats["sem"],
    fmt='o-',                     # line + circle markers
    color='red',
    ecolor='black',               # color of error bars
    elinewidth=1,
    capsize=4                     # small caps on error bars
)

# Step 6: Labeling and formatting
plt.xticks(ticks=x_positions, labels=bin_labels, rotation=45)
plt.xlabel("Num. features same")
plt.ylabel("Mean log(RT)")
# plt.title("Mean log(RT) with Standard Error across Quantile Bins")
plt.tight_layout()
plt.show()


In [ ]:
def get_dot_product(df, featurecols):
    df['dot_product'] = None
    df = df.groupby('pid', group_keys=False).apply(calculate_dot_product, featurecols = featurecols)
    return df

def calculate_dot_product(group, featurecols):
    group = group.reset_index(drop=True)
    dot_product = [np.nan, np.nan]  # Initialize with nan for the first row
    
    for i in range(2, len(group)):
        row1 = group.loc[i - 2, featurecols]
        row2 = group.loc[i - 1, featurecols]
        row3 = group.loc[i, featurecols]
        
        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            dot_product.append(np.nan)
        else:
            vec1 = (row1 == row2).astype(int)
            vec2 = (row2 == row3).astype(int)
            dot_product.append(np.dot(vec1, vec2))
    
    group['dot_product'] = dot_product  
    return group

# data_vf = get_dot_product(data_vf, vf_featurecols)
data2 = get_dot_product(data2, vf_featurecols)
# data3 = get_dot_product(data3, vf_featurecols)
# data4 = get_dot_product(data4, vf_featurecols)
# data5 = get_dot_product(data5, vf_featurecols)

In [ ]:
def calculate_dot_product(group, featurecols):
    group = group.reset_index(drop=True)
    n = len(group)
    
    if n < 3:
        group['dot_product'] = [np.nan] * n
        return group
    
    dot_product = [np.nan, np.nan]  # For first two rows

    for i in range(2, n):
        row1 = group.loc[i - 2, featurecols]
        row2 = group.loc[i - 1, featurecols]
        row3 = group.loc[i, featurecols]

        if row1.isna().any() or row2.isna().any():
            dot_product.append(np.nan)
        else:
            vec1 = (row1 == row2).astype(int)
            vec2 = (row2 == row3).astype(int)
            dot_product.append(np.dot(vec1, vec2))

    group['dot_product'] = dot_product
    return group

# data_vf = get_dot_product(data_vf, vf_featurecols)
# data2 = get_dot_product(data2, vf_featurecols)
data3 = get_dot_product(data3, vf_featurecols)
data4 = get_dot_product(data4, vf_featurecols)
data5 = get_dot_product(data5, vf_featurecols)

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(data3["num_features_same"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="noconstraints")
plt.hist(data4["num_features_same"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="similar")
plt.hist(data5["num_features_same"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="divergent")
plt.legend();

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(data3["dot_product"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="noconstraints")
plt.hist(data4["dot_product"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="similar")
plt.hist(data5["dot_product"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="divergent")
plt.legend();

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(data3["RT"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="noconstraints", bins=100)
plt.hist(data4["RT"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="similar", bins=100)
plt.hist(data5["RT"].tolist()[:min(len(data3), len(data4), len(data5))], alpha=0.3, label="divergent", bins=100)
plt.xlim(0, 20000)
plt.legend();

In [ ]:
plt.figure(figsize=(6,5))
plt.hist(data3.groupby("pid").count()["rid"].tolist(), alpha=0.3, label="noconstraints", bins=10)
plt.hist(data4.groupby("pid").count()["rid"].tolist(), alpha=0.3, label="similar", bins=10)
plt.hist(data5.groupby("pid").count()["rid"].tolist(), alpha=0.3, label="divergent", bins=10)
plt.legend();

In [ ]:
data4[["RT", "num_features_same", "dot_product"]].corr()

In [ ]:
from scipy.stats import spearmanr

from scipy.stats import spearmanr

# Data 2
subset = data2[["num_features_same", "dot_product", "RT"]].dropna()
# print("data2 - Pearson:",
#       np.corrcoef(subset["num_features_same"], subset["RT"])[0, 1],
#       np.corrcoef(subset["dot_product"], subset["RT"])[0, 1])
print("data2 - Spearman:",
      spearmanr(subset["num_features_same"], subset["RT"]).correlation,
      spearmanr(subset["dot_product"], subset["RT"]).correlation)

# Data 3
subset = data3[["num_features_same", "dot_product", "RT"]].dropna()
# print("data3 - Pearson:",
#       np.corrcoef(subset["num_features_same"], subset["RT"])[0, 1],
#       np.corrcoef(subset["dot_product"], subset["RT"])[0, 1])
print("data3 - Spearman:",
      spearmanr(subset["num_features_same"], subset["RT"]).correlation,
      spearmanr(subset["dot_product"], subset["RT"]).correlation)

# Data 4
subset = data4[["num_features_same", "dot_product", "RT"]].dropna()
# print("data4 - Pearson:",
#       np.corrcoef(subset["num_features_same"], subset["RT"])[0, 1],
#       np.corrcoef(subset["dot_product"], subset["RT"])[0, 1])
print("data4 - Spearman:",
      spearmanr(subset["num_features_same"], subset["RT"]).correlation,
      spearmanr(subset["dot_product"], subset["RT"]).correlation)

# Data 5
subset = data5[["num_features_same", "dot_product", "RT"]].dropna()
# print("data5 - Pearson:",
#       np.corrcoef(subset["num_features_same"], subset["RT"])[0, 1],
#       np.corrcoef(subset["dot_product"], subset["RT"])[0, 1])
print("data5 - Spearman:",
      spearmanr(subset["num_features_same"], subset["RT"]).correlation,
      spearmanr(subset["dot_product"], subset["RT"]).correlation)


In [ ]:
subset = data2[["num_features_same", "dot_product"]].dropna()

In [ ]:
data2['num_features_same'].corr(data2['dot_product'], method='pearson')

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["num_features_same"].tolist())
plt.ylabel("Number of Ppts")
plt.xlabel("Mean num features same")
plt.xlim(85, 110);

plt.figure(figsize=(3,3))
plt.hist(data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["dot_product"].tolist())
plt.ylabel("Number of Ppts")
plt.xlabel("Mean dot product")
plt.xlim(65, 95);

In [ ]:
np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values)

In [ ]:
# shuffle within each pid
from tqdm import tqdm
means = []
for _ in tqdm(range(100)):
    shuffled_data2 = data2.groupby("pid", group_keys=False).apply(lambda x: x.sample(frac=1).reset_index(drop=True))
    shuffled_data2 = get_num_features_same(shuffled_data2, vf_featurecols)
    means.append(np.mean(shuffled_data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values))
# shuffled_data2 = get_dot_product(shuffled_data2, vf_featurecols)

In [ ]:
plt.figure(figsize=(4,4))
plt.hist(means, bins=6, label="Shuffled")
# plt.axvline(np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values), color='red')
plt.ylabel("Number of simulations")
plt.xlabel("Mean num features same");

In [ ]:
from brokenaxes import brokenaxes
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 3.5))

# Example histogram data
plt.rcParams['figure.dpi'] = 150
# means = [98.2, 98.3, 98.4, 98.6, 98.7, 98.5] * 10  # your simulated data

# Create a broken x-axis: main region (98.1–98.9) and jump to (105.5–106.5)
bax = brokenaxes(xlims=((98.1, 98.9), (np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values) - 0.1, np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values) + 0.1)), hspace=.05)

# Plot histogram in the first x-range
bax.hist(means, bins=6, color='lightcoral', label="Shuffled")

# Add vertical line at 106 in the second x-range
bax.axvline(np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values), color='red', linestyle='--', linewidth=2, label=f'  Data =\n  {str(np.round(np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values),2))}')
# bax.text(np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values), 8, f'  Data =\n  {str(np.round(np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values),2))}', color='red', fontsize=8)

# Label axes
bax.set_xlabel("Mean num features same", labelpad=25)
bax.set_ylabel("Number of simulations", labelpad=25)

make plt.xticks([97, 97, 99 // 105 106])
show plt.legend()

plt.show()

In [ ]:
from brokenaxes import brokenaxes
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.dpi'] = 150
plt.figure(figsize=(4, 3.5))

# Calculate data mean
data_mean = np.mean(data2[["num_features_same", "pid"]].groupby("pid").mean()["num_features_same"].values)

# Create broken x-axis
bax = brokenaxes(
    xlims=((98, 99), (data_mean - 0.1, data_mean + 0.1)),
    hspace=0.05
)

# Plot histogram
bax.hist(means, bins=6, color='lightcoral', label="Shuffled")

# Add vertical line for observed data
bax.axvline(data_mean, color='red', linestyle='--', linewidth=2, label=f'Data = {data_mean:.2f}')

# Axis labels
bax.set_xlabel("Mean num features same", labelpad=25)
bax.set_ylabel("Number of simulations", labelpad=25)

# Custom x-ticks
bax.axs[0].set_xticks([98.2, 98.5, 98.8])
bax.axs[1].set_xticks([105, 106])  # or [round(data_mean)] if dynamic

# Add legend
bax.legend(loc='upper right')

plt.show()

In [ ]:
plt.figure(figsize=(3,3))
plt.hist(shuffled_data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["num_features_same"].tolist())
plt.ylabel("Number of Ppts")
plt.xlabel("Mean num features same")
plt.xlim(85, 110);

plt.figure(figsize=(3,3))
plt.hist(shuffled_data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["dot_product"].tolist())
plt.ylabel("Number of Ppts")
plt.xlabel("Mean dot product")
plt.xlim(65, 95);

In [ ]:
plt.rcParams.update({
    "axes.facecolor": "white",                  # background stays white
    "axes.edgecolor": "black",                  # keep axis edges
    "patch.facecolor": "lightcoral",
    "text.usetex": False,                     # render text with LaTeX
    "font.family": "sans-serif",                  # use serif fonts
    "axes.spines.top": False,                # remove top border
    "axes.spines.right": False,              # remove right border
    "axes.labelsize": 16,                    # bigger axis labels
    "xtick.labelsize": 14,                   # bigger x-tick labels
    "ytick.labelsize": 14,                   # bigger y-tick labels
    "axes.titlesize": 18,                    # bigger title
    "figure.dpi": 120,                       # higher resolution
})

plt.figure(figsize=(4,4))
plt.hist(data2[["pid", "num_features_same", "dot_product"]]["num_features_same"].tolist(), alpha=0.5, color="mediumaquamarine", label="Original data")
plt.ylabel("Num. response-pairs")
plt.xlabel("Num. features same")

plt.hist(shuffled_data2[["pid", "num_features_same", "dot_product"]]["num_features_same"].tolist(), alpha=0.5, color="slateblue", label="Shuffled data")
plt.ylabel("Num. response-pairs")
plt.xlabel("Num. features same")

plt.legend()

In [ ]:
plt.figure(figsize=(4,4))
plt.hist(data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["num_features_same"].tolist(), alpha=0.5, color="mediumaquamarine", label="Original data")
plt.ylabel("Num. participants")
plt.xlabel("Mean num. features same")

plt.hist(shuffled_data2[["pid", "num_features_same", "dot_product"]].groupby("pid").mean()["num_features_same"].tolist(), alpha=0.5, color="slateblue", label="Shuffled data")
plt.ylabel("Num. participants")
plt.xlabel("Mean num. features same")
plt.legend()

In [ ]:
def get_num_features_same(df, featurecols):
    df['num_features_same_2back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(2, len(group)):
        row1 = group.loc[i - 2, featurecols]
        row2 = group.loc[i, featurecols]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            consecutive_1s = ((row1 == 1) & (row2 == 1)) | ((row1 == 0) & (row2 == 0))
            num_features_same[i] = consecutive_1s.sum()
    
    group['num_features_same_2back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
# data_vf = get_num_features_same(data_vf, vf_featurecols)
data2 = get_num_features_same(data2, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['num_features_same_3back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(3, len(group)):
        row1 = group.loc[i - 3, featurecols]
        row2 = group.loc[i, featurecols]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            consecutive_1s = ((row1 == 1) & (row2 == 1)) | ((row1 == 0) & (row2 == 0))
            num_features_same[i] = consecutive_1s.sum()
    
    group['num_features_same_3back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
# data_vf = get_num_features_same(data_vf, vf_featurecols)
data2 = get_num_features_same(data2, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['num_features_same_4back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(4, len(group)):
        row1 = group.loc[i - 4, featurecols]
        row2 = group.loc[i, featurecols]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            consecutive_1s = ((row1 == 1) & (row2 == 1)) | ((row1 == 0) & (row2 == 0))
            num_features_same[i] = consecutive_1s.sum()
    
    group['num_features_same_4back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
# data_vf = get_num_features_same(data_vf, vf_featurecols)
data2 = get_num_features_same(data2, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['num_features_same_5back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(5, len(group)):
        row1 = group.loc[i - 5, featurecols]
        row2 = group.loc[i, featurecols]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            consecutive_1s = ((row1 == 1) & (row2 == 1)) | ((row1 == 0) & (row2 == 0))
            num_features_same[i] = consecutive_1s.sum()
    
    group['num_features_same_5back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
# data_vf = get_num_features_same(data_vf, vf_featurecols)
data2 = get_num_features_same(data2, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
# Calculate means and standard errors
means = [
    np.mean(data2["num_features_same_5back"]),
    np.mean(data2["num_features_same_4back"]),
    np.mean(data2["num_features_same_3back"]),
    np.mean(data2["num_features_same_2back"]),
    np.mean(data2["num_features_same"])
]

std_errors = [
    np.std(data2["num_features_same_5back"], ddof=1) / np.sqrt(len(data2["num_features_same_5back"].dropna())),
    np.std(data2["num_features_same_4back"], ddof=1) / np.sqrt(len(data2["num_features_same_4back"].dropna())),
    np.std(data2["num_features_same_3back"], ddof=1) / np.sqrt(len(data2["num_features_same_3back"].dropna())),
    np.std(data2["num_features_same_2back"], ddof=1) / np.sqrt(len(data2["num_features_same_2back"].dropna())),
    np.std(data2["num_features_same"], ddof=1) / np.sqrt(len(data2["num_features_same"].dropna()))
]

x_labels = [-5, -4, -3, -2, -1]

# Plot bar chart with error bars
plt.figure(figsize=(4.55, 4.55))
plt.bar(x_labels, means, yerr=std_errors, capsize=5, alpha=0.7, color='lightcoral')
plt.xlabel("Pos. preceeding most recent resp.")
plt.ylabel("Mean num. features same")
plt.ylim(95, 107)
plt.xticks(x_labels)
plt.show()

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_1back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(1, len(group)):
        row1 = group.loc[i - 1, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_1back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_2back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(2, len(group)):
        row1 = group.loc[i - 2, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_2back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_3back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(3, len(group)):
        row1 = group.loc[i - 3, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_3back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_4back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(4, len(group)):
        row1 = group.loc[i - 4, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_4back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_5back'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(5, len(group)):
        row1 = group.loc[i - 5, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_5back'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_0'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(0, len(group)):
        row1 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = 0
            num_features_same[i] = RTdiff
    
    group['RT_diff_0'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_1ahead'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(0, len(group) - 1):
        row1 = group.loc[i + 1, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_1ahead'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
def get_num_features_same(df, featurecols):
    df['RT_diff_2ahead'] = np.nan  # Initialize the column with NaN
    df = df.groupby('pid', group_keys=False).apply(calculate_num_features_same, featurecols=featurecols)
    return df

def calculate_num_features_same(group, featurecols):
    group = group.reset_index(drop=True)
    
    num_features_same = np.full(len(group), np.nan)  # Initialize with NaN
    
    for i in range(0, len(group) - 2):
        row1 = group.loc[i + 2, ["RT"]]
        row2 = group.loc[i, ["RT"]]

        # Check for NaN values
        if row1.isna().any() or row2.isna().any():
            num_features_same[i] = np.nan
        else:
            RTdiff = row2["RT"] - row1["RT"]
            num_features_same[i] = RTdiff
    
    group['RT_diff_2ahead'] = num_features_same  # Assign correctly
    return group

# Apply the function to your datasets
data_vf = get_num_features_same(data_vf, vf_featurecols)
# data_autbrick = get_num_features_same(data_autbrick, autbrick_featurecols)
# data_autpaperclip = get_num_features_same(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
# Calculate means and standard errors
means = [
    np.mean(data_vf["RT_diff_5back"]),
    np.mean(data_vf["RT_diff_4back"]),
    np.mean(data_vf["RT_diff_3back"]),
    np.mean(data_vf["RT_diff_2back"]),
    np.mean(data_vf["RT_diff_1back"]),
    np.mean(data_vf["RT_diff_0"]),
    np.mean(data_vf["RT_diff_1ahead"]),
    np.mean(data_vf["RT_diff_2ahead"])
]

std_errors = [
    np.std(data_vf["RT_diff_5back"], ddof=1) / np.sqrt(len(data_vf["RT_diff_5back"].dropna())),
    np.std(data_vf["RT_diff_4back"], ddof=1) / np.sqrt(len(data_vf["RT_diff_4back"].dropna())),
    np.std(data_vf["RT_diff_3back"], ddof=1) / np.sqrt(len(data_vf["RT_diff_3back"].dropna())),
    np.std(data_vf["RT_diff_2back"], ddof=1) / np.sqrt(len(data_vf["RT_diff_2back"].dropna())),
    np.std(data_vf["RT_diff_1back"], ddof=1) / np.sqrt(len(data_vf["RT_diff_1back"].dropna())),
    np.std(data_vf["RT_diff_0"], ddof=1) / np.sqrt(len(data_vf["RT_diff_0"].dropna())),
    np.std(data_vf["RT_diff_1ahead"], ddof=1) / np.sqrt(len(data_vf["RT_diff_1ahead"].dropna())),
    np.std(data_vf["RT_diff_2ahead"], ddof=1) / np.sqrt(len(data_vf["RT_diff_2ahead"].dropna()))
]

x_labels = [-5, -4, -3, -2, -1, 0, 1, 2]

# Plot bar chart with error bars
plt.figure(figsize=(8, 6))
plt.bar(x_labels, means, yerr=std_errors, capsize=5, alpha=0.7, color='mediumpurple')
plt.xlabel("Back Steps")
plt.ylabel("Mean RT")
plt.title("Mean RT")
plt.show()

In [ ]:
def has_two_chunks(lst):
    count = 0  # Count of contiguous chunks of ones with length > 1
    i = 0
    while i < len(lst):
        if lst[i] == 1:
            start = i
            while i < len(lst) and lst[i] == 1:
                i += 1
            if i - start > 1:
                count += 1
                if count >= 2:
                    return 1
        else:
            i += 1
    return 0

# Apply function to each feature_* column, grouped by 'pid'
returned_to_same_feature = {
    col: data_vf.groupby("pid")[col].apply(has_two_chunks) for col in vf_featurecols
}

# Convert results to DataFrame
returned_to_same_feature_df = pd.DataFrame(returned_to_same_feature)
returned = (np.sum(returned_to_same_feature_df, axis=0)/len(data_vf["pid"].unique())).to_dict()
returned = dict(sorted(returned.items(), key=lambda item: item[1]))
plt.figure(figsize=(3,14))
plt.barh(list(returned.keys()), list(returned.values()))
plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(3,2))
plt.hist(data_vf["num_features_same"], alpha=0.3, label="vf", color="mediumpurple");
# plt.hist(data_autbrick["num_features_same"], alpha=0.3, label="autbrick");
# plt.hist(data_autpaperclip["num_features_same"], alpha=0.3, label="autpaperclip");
# plt.legend();
plt.xlabel("Number of features same")
plt.ylabel("Num responses")

In [ ]:
data_vf.head(30)

In [ ]:
data_vf = data_vf[(data_vf["order"] > 0) & (data_vf["order"] < 20)]
# data_autbrick = data_autbrick[(data_autbrick["order"] > 0) & (data_autbrick["order"] < 20)]
# data_autpaperclip = data_autpaperclip[(data_autpaperclip["order"] > 0) & (data_autpaperclip["order"] < 20)]

In [ ]:
data_humans = pd.read_csv("../csvs/data_humans.csv")

In [ ]:
data_vf.columns

In [ ]:
data_vf = pd.merge(data_vf, data_humans[data_humans["task"] == 1].drop("Unnamed: 0", axis=1), on=['pid', 'task', 'starttime', 'endtime', 'original_response_Dutch',
       'original_response', 'original_response_cleaned', 'response', 'invalid', 'response_len', 'response_num_words', 'order', 'RT'], how='left')
data_vf = data_vf[~data_vf["response"].isin(vf_to_remove)]

# data_autbrick = pd.merge(data_autbrick, data_humans[data_humans["task"] == 2].drop("Unnamed: 0", axis=1), on=['pid', 'task', 'starttime', 'endtime', 'original_response_Dutch',
#        'original_response', 'original_response_cleaned', 'response', 'invalid', 'response_len', 'response_num_words', 'previous_original_response', 'previous_response', 'order', 'RT'], how='left')
# data_autpaperclip = pd.merge(data_autpaperclip, data_humans[data_humans["task"] == 3].drop("Unnamed: 0", axis=1), on=['pid', 'task', 'starttime', 'endtime', 'original_response_Dutch',
#        'original_response', 'original_response_cleaned', 'response', 'invalid', 'response_len', 'response_num_words', 'previous_original_response', 'previous_response', 'order', 'RT'], how='left')

In [ ]:
plt.figure(figsize=(4,3))
valid_indices = ~np.isnan(data_vf["SS"]) & ~np.isnan(data_vf["num_features_same"])
filtered_SS = data_vf["SS"][valid_indices]
filtered_num_features_same = data_vf["num_features_same"][valid_indices]
plt.scatter(filtered_SS, filtered_num_features_same, alpha=0.3, c="mediumpurple")
print(np.corrcoef(filtered_SS, filtered_num_features_same)[0,1])
plt.xlabel("GTE Large SS")
plt.ylabel("Number of features same");

# plt.figure()
# plt.scatter(data_autbrick["SS"], data_autbrick["num_features_same"])
# plt.xlabel("SS")
# plt.ylabel("Number of features same");

# plt.figure()
# plt.scatter(data_autpaperclip["SS"], data_autpaperclip["num_features_same"])
# plt.xlabel("SS")
# plt.ylabel("Number of features same");

In [ ]:
data_vf['previous_response'] = data_vf.groupby('pid')['response'].shift(1)

In [ ]:
# least similar
# print(data_vf[["response", "previous_response", "num_features_same", "SS", "jump"]][data_vf["num_features_same"] < 10]["SS"].mean())
data_vf[["pid", "response", "previous_response", "num_features_same"]][data_vf["num_features_same"] < 44].drop_duplicates().head(50)

In [ ]:
# most similar
# print(data_vf[["response", "previous_response", "num_features_same", "SS", "jump"]][data_vf["num_features_same"] > 80]["SS"].mean())
data_vf[["response", "previous_response", "num_features_same"]][data_vf["num_features_same"] > 78].drop_duplicates().sort_values(by=["num_features_same"], ascending=False).head(50)

In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Step 1: Assume your dataframe is called df
names = vf_featuredf.iloc[:, 0]            # First column = names
features = vf_featuredf.iloc[:, 1:]        # Rest = binary features

# Step 2: t-SNE with Hamming distance (good for binary vectors)
tsne = TSNE(n_components=2, perplexity=50, random_state=42, metric='hamming')
embedding = tsne.fit_transform(features)

# Step 3: Plotting
plt.figure(figsize=(10, 7))
plt.scatter(embedding[:, 0], embedding[:, 1], s=50, alpha=0.5, c="mediumpurple")

# Optionally annotate points with names
for i, name in enumerate(names):
    if i % 4 == 0:
        plt.text(embedding[i, 0] + 0.005, embedding[i, 1] + 0.005, str(name), fontsize=10)

plt.title("t-SNE of Binary Vectors")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.grid(False)
plt.show()

In [ ]:
vf_featuredf = vf_featuredf.reset_index()

In [ ]:
temp = np.array(list(pd.merge(data_vf[["pid", "num_features_same"]].groupby("pid").mean().reset_index(), data_vf[["pid", "jump_profile"]].groupby("pid").max().reset_index(), on=["pid"]).sort_values("jump_profile")["num_features_same"]))
arr = temp[np.argsort(temp)][:-1]

In [ ]:
# plt.scatter(data_vf[["pid", "jump_profile"]].groupby("pid").max().reset_index()["jump_profile"].values, data_vf[["pid", "num_features_same"]].groupby("pid").mean()["num_features_same"].values)
plt.figure(figsize=(4,3))
plt.scatter(np.arange(219), pd.merge(data_vf[["pid", "num_features_same"]].groupby("pid").mean().reset_index(), data_vf[["pid", "jump_profile"]].groupby("pid").max().reset_index(), on=["pid"]).sort_values("jump_profile")["num_features_same"], c="mediumpurple")
print(np.corrcoef(np.arange(218), arr)[0,1])
plt.xlabel("Max jump profile")
plt.ylabel("Mean features same");

# plt.figure()
# t = pd.merge(data_autbrick[["pid", "num_features_same"]].groupby("pid").mean().reset_index(), data_autbrick[["pid", "jump_profile"]].groupby("pid").max().reset_index(), on=["pid"]).sort_values("jump_profile")["num_features_same"]
# plt.scatter(np.arange(len(t)), t)
# plt.xlabel("Max jump profile")
# plt.ylabel("Mean features same");

# plt.figure()
# t = pd.merge(data_autpaperclip[["pid", "num_features_same"]].groupby("pid").mean().reset_index(), data_autpaperclip[["pid", "jump_profile"]].groupby("pid").max().reset_index(), on=["pid"]).sort_values("jump_profile")["num_features_same"]
# plt.scatter(np.arange(len(t)), t)
# plt.xlabel("Max jump profile")
# plt.ylabel("Mean features same");

In [ ]:
def plot_binary_features_heatmap(df, featurecols):
    pids = df["pid"].unique()
    cnt = 1
    for pid in pids:
        responses = df[df["pid"] == pid]["response"].values
        pid_data = df[df["pid"] == pid][featurecols].reset_index(drop=True)
        
        plt.figure(figsize=(25, len(pid_data) * 0.25))
        sns.heatmap(
            pid_data,
            cmap=sns.color_palette(["white", "mediumpurple"]),
            cbar=False,
            linewidths=0.5,
            linecolor='black'
        )
        plt.title(f"Binary Features Heatmap for PID {pid}")
        plt.gca().xaxis.tick_top()
        plt.xticks(ticks=np.arange(len(featurecols)) + 0.5, labels=featurecols, rotation=90)
        plt.yticks(ticks=np.arange(len(responses)) + 0.5, labels=responses, rotation=0)
        plt.show()
        
        if cnt == 10:
            break
        cnt += 1

In [ ]:
data_vf

In [ ]:
plot_binary_features_heatmap(data_vf, vf_featurecols)

In [ ]:
data_vf

In [ ]:
plot_binary_features_heatmap(data_autbrick, autbrick_featurecols)

In [ ]:
plot_binary_features_heatmap(data_autpaperclip, autpaperclip_featurecols)

In [ ]:
# Few vs many dimensions
# How persistant are different dimensions? Compared to random
# Transitions between different dimensions? 